# The Insecure Code

In [12]:
# ----------------- ⚠️ INSECURE PART ⚠️ -----------------
# Hard-coding credentials directly in the code. Do NOT do this in production!
tenant_id = "xxxxxxxxxxxxxx"
client_id = "xxxxxxxxxxxxxx"
client_secret = "xxxxxxxxxxxxxx"

StatementMeta(, 7f421df3-5c3b-4da5-b9c5-1b68bc4b4d77, 35, Finished, Available, Finished)

In [ ]:
# Step 1: Install the necessary libraries once per kernel run
%pip install msal requests

# Insecure Method

In [16]:
# Step 2: Import the required libraries
import msal
import requests
import json

# ---------------------------------------------------------

# Step 3: Authenticate and get an access token using MSAL
authority = f"https://login.microsoftonline.com/{tenant_id}"
scope = ["https://analysis.windows.net/powerbi/api/.default"]

app = msal.ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)
result = app.acquire_token_for_client(scopes=scope)

if "access_token" in result:
    access_token = result['access_token']
    print("✅ MSAL authentication successful!")

    # Step 4: Use the access token to call the Power BI REST API
    api_url = "https://api.powerbi.com/v1.0/myorg/groups" # This endpoint gets workspaces
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    
    response = requests.get(api_url, headers=headers)
    
    if response.status_code == 200:
        workspaces = response.json().get('value', [])
        print("Workspaces this Service Principal can access:")
        for ws in workspaces:
            print(f"- {ws['name']} (ID: {ws['id']})")
    else:
        print(f"❌ API call failed with status {response.status_code}: {response.text}")
else:
    print(f"❌ MSAL authentication failed: {result.get('error_description')}")

StatementMeta(, 7f421df3-5c3b-4da5-b9c5-1b68bc4b4d77, 39, Finished, Available, Finished)

✅ MSAL authentication successful!
Workspaces this Service Principal can access:
- MOME_2022_Oct (ID: d6e64a98-9439-44f5-8523-d4cf8046e56b)
- Fabric_Dojo (ID: d22e55d7-5d85-4c31-8023-697049907e7f)
- Columny (ID: aa9c9417-a807-4e1a-a7e4-04bcfa61b615)


In [17]:
response.json()

StatementMeta(, 7f421df3-5c3b-4da5-b9c5-1b68bc4b4d77, 40, Finished, Available, Finished)

{'@odata.context': 'https://wabi-west-europe-e-primary-redirect.analysis.windows.net/v1.0/myorg/$metadata#groups',
 '@odata.count': 3,
 'value': [{'id': 'd6e64a98-9439-44f5-8523-d4cf8046e56b',
   'isReadOnly': False,
   'isOnDedicatedCapacity': True,
   'capacityId': '314E69EE-913F-423E-BB95-DA355B853BC0',
   'defaultDatasetStorageFormat': 'Large',
   'type': 'Workspace',
   'name': 'MOME_2022_Oct'},
  {'id': 'd22e55d7-5d85-4c31-8023-697049907e7f',
   'isReadOnly': False,
   'isOnDedicatedCapacity': True,
   'capacityId': '314E69EE-913F-423E-BB95-DA355B853BC0',
   'defaultDatasetStorageFormat': 'Small',
   'type': 'Workspace',
   'name': 'Fabric_Dojo'},
  {'id': 'aa9c9417-a807-4e1a-a7e4-04bcfa61b615',
   'isReadOnly': False,
   'isOnDedicatedCapacity': True,
   'capacityId': '314E69EE-913F-423E-BB95-DA355B853BC0',
   'defaultDatasetStorageFormat': 'Small',
   'type': 'Workspace',
   'name': 'Columny'}]}

# Method 2: The Secure & Professional Way (Using Azure Key Vault) - The Final Version

In [3]:
# Step 2: Import the required libraries
import msal
import requests
import json
from notebookutils import mssparkutils

# Step 3: Define the location of your Key Vault
# ----------------- ✅ CORRECTION HERE ✅ -----------------
# The function requires the FULL Vault URI, not just the name.
# Get this from the "Vault URI" property on your Key Vault's overview page in the Azure Portal.
key_vault_uri = "https://vaultyberty.vault.azure.net/" 
# --------------------------------------------------------

# Step 4: Securely retrieve secrets from Azure Key Vault
print("Attempting to retrieve secrets from Azure Key Vault...")
try:
    # NOTE: The first parameter is now the full URI
    tenant_id = mssparkutils.credentials.getSecret(key_vault_uri, "spn-tenant-id")
    client_id = mssparkutils.credentials.getSecret(key_vault_uri, "spn-client-id")
    client_secret = mssparkutils.credentials.getSecret(key_vault_uri, "spn-client-secret")
    print("✅ Successfully retrieved secrets from Key Vault.")

    # Step 5: Authenticate and get an access token using MSAL
    authority = f"https://login.microsoftonline.com/{tenant_id}"
    scope = ["https://analysis.windows.net/powerbi/api/.default"]

    app = msal.ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)
    result = app.acquire_token_for_client(scopes=scope)

    if "access_token" in result:
        access_token = result['access_token']
        print("✅ MSAL authentication successful using credentials from Key Vault!")

        # Step 6: Use the access token to call the Power BI REST API
        api_url = "https://api.powerbi.com/v1.0/myorg/groups"
        headers = {'Authorization': f'Bearer {access_token}'}
        
        response = requests.get(api_url, headers=headers)
        
        if response.status_code == 200:
            workspaces = response.json().get('value', [])
            print("Workspaces this Service Principal can access:")
            for ws in workspaces:
                print(f"- {ws['name']} (ID: {ws['id']})")
        else:
            print(f"❌ API call failed with status {response.status_code}: {response.text}")
    else:
        print(f"❌ MSAL authentication failed: {result.get('error_description')}")

except Exception as e:
    # This will catch errors from both Key Vault and the subsequent code
    print(f"❌ An error occurred. Check vault URI and permissions. Error: {e}")

StatementMeta(, d7c37fcc-51d5-40dc-8317-c0caf50de763, 11, Finished, Available, Finished)

Attempting to retrieve secrets from Azure Key Vault...
✅ Successfully retrieved secrets from Key Vault.
✅ MSAL authentication successful using credentials from Key Vault!
Workspaces this Service Principal can access:
- MOME_2022_Oct (ID: d6e64a98-9439-44f5-8523-d4cf8046e56b)
- Fabric_Dojo (ID: d22e55d7-5d85-4c31-8023-697049907e7f)
- Columny (ID: aa9c9417-a807-4e1a-a7e4-04bcfa61b615)
